In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler, KMeansSMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
from datetime import timedelta


from ADVO.generator import Generator
from ADVO.oversampler import ADVO, TimeGANOverSampler, CTGANOverSampler
from ADVO.utils import evaluate_models, compute_kde_difference_auc

SAMPLE_STRATEGY = 0.18
N_JOBS = 6
N_TREES = 20
N_USERS = 10000
N_TERMINALS = 1000
RANDOM_STATE = 42

RANDOM_GRID_RF = {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_features': [1, 'sqrt', 'log2'], 'max_depth': [5, 16, 28, 40, None], 'min_samples_split': [10, 25, 50], 'min_samples_leaf': [4, 8, 32], 'bootstrap': [True, False]}
RANDOM_GRID_RIDGE = {'alpha': [int(x) for x in np.linspace(start = 0.001, stop = 1, num = 100)], 'fit_intercept': [True, False]}
RANDOM_GRID_NN = {'hidden_layer_sizes': [int(x) for x in np.linspace(start = 1, stop = 41, num = 80)], 'alpha': [int(x) for x in np.linspace(start = 0.005, stop = 0.02, num = 100)]}


CANDIDATE_REGRESSORS = [MLPRegressor(max_iter=2000, random_state=RANDOM_STATE), Ridge(random_state=RANDOM_STATE), RandomForestRegressor(random_state=RANDOM_STATE)]
CANDIDATE_GRIDS = [RANDOM_GRID_NN, RANDOM_GRID_RIDGE, RANDOM_GRID_RF]

def fit_predict(X_train,y_train,learner, X_test, predictions_proba, discrete_predictions):
    learner.fit(X_train, y_train)
    y_hat = learner.predict(X_test)
    y_hat_proba = learner.predict_proba(X_test)[:,1]
    predictions_proba.append(y_hat_proba)
    discrete_predictions.append(y_hat)

def run_advo(X_train, y_train, window_counter):
    advo = ADVO(n_jobs=N_JOBS,sampling_strategy=SAMPLE_STRATEGY,random_state=RANDOM_STATE, mimo=False)
    advo.set_transactions(X_train, y_train)
    advo.create_couples()
    regressor_scores = advo.select_best_regressor(candidate_regressors=CANDIDATE_REGRESSORS,parameters_set=CANDIDATE_GRIDS)
    advo.tune_best_regressors()
    advo.fit_regressors()
    advo.transactions_df = advo.insert_synthetic_frauds(advo.transactions_df)
    regressor_scores.to_csv('results/regressor_scores_'+str(window_counter)+'.csv', index=False)
    return advo


c:\ProgramData\Anaconda3\envs\advo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_size_days=20
test_size_days=2

In [4]:
#transactions_df = Generator().generate(filename='dataset_six_months.csv',nb_days_to_generate=180)
transactions_df = pd.read_csv('utils/dataset_six_months.csv', parse_dates=['TX_DATETIME'])

start_date, end_date = transactions_df['TX_DATETIME'].min(), transactions_df['TX_DATETIME'].max()

window_start, window_end, window_counter  = start_date, start_date + timedelta(days=train_size_days), 0


# Split data into train and test according to the window
train_mask, test_mask = (transactions_df['TX_DATETIME'] >= window_start) & (transactions_df['TX_DATETIME'] < window_end), (transactions_df['TX_DATETIME'] >= window_end) & (transactions_df['TX_DATETIME'] < window_end + timedelta(days=test_size_days))
X_train, y_train, X_test, y_test = transactions_df[train_mask].drop(columns=['TX_FRAUD']), transactions_df[train_mask]['TX_FRAUD'], transactions_df[test_mask].drop(columns=['TX_FRAUD']), transactions_df[test_mask]['TX_FRAUD']
training_variables, predictions_proba, discrete_predictions = ['X_TERMINAL', 'Y_TERMINAL', 'TX_AMOUNT'], [], []

In [5]:
timegan = TimeGANOverSampler(sampling_strategy=SAMPLE_STRATEGY, epochs=300, seq_len=4, n_seq=3, hidden_dim=24, gamma=1, noise_dim = 32, dim = 128, batch_size = 32, log_step = 100, learning_rate = 5e-4,random_state=RANDOM_STATE).fit_resample(X_train[training_variables+['CUSTOMER_ID']], y_train)

c:\Users\Administrateur\git\ADV-O\ADVO\oversampler\timegan_wrapper.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frauds_df.drop(columns=['TX_FRAUD'], inplace=True)
Synthetic data generation: 100%|██████████| 9/9 [00:01<00:00,  8.93it/s]
c:\Users\Administrateur\git\ADV-O\ADVO\oversampler\timegan_wrapper.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  synthetic_data = synthetic_data.append(pd.DataFrame(synth_data[i]), ignore_index=True)


In [6]:
timegan

(       X_TERMINAL  Y_TERMINAL   TX_AMOUNT
 0        0.999818    0.999432    0.999474
 1        0.999869    0.999578    0.999613
 2        0.999870    0.999580    0.999615
 3        0.999870    0.999580    0.999615
 4        0.999818    0.999432    0.999474
 ...           ...         ...         ...
 76098   90.830000   23.960000  143.730000
 76099   77.220000   19.870000  123.960000
 76100   94.290000   32.320000   64.070000
 76101   77.220000   19.870000  111.170000
 76102   83.240000   21.230000   90.900000
 
 [9702 rows x 3 columns],
 0        1
 1        1
 2        1
 3        1
 4        1
         ..
 76098    0
 76099    0
 76100    0
 76101    0
 76102    0
 Name: TX_FRAUD, Length: 9702, dtype: int64,
       X_TERMINAL  Y_TERMINAL  TX_AMOUNT  TX_FRAUD
 0       0.999818    0.999432   0.999474         1
 1       0.999869    0.999578   0.999613         1
 2       0.999870    0.999580   0.999615         1
 3       0.999870    0.999580   0.999615         1
 4       0.999818    0.9